In [1]:
pip install pyarrow geopandas geoarrow-pyarrow geoarrow-pandas -q


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import geopandas as gpd

# File Import Tests

You can download the two files in this notebook here:

- [2015 Tree Census](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/pi5s-9p35)
- [2010 Census Block Groups](https://data.cityofnewyork.us/City-Government/2010-Census-Blocks/v2h8-6mxf)

In [3]:
%%time

gdf = gpd.read_file('./trees.geojson')

CPU times: user 3min 19s, sys: 1min, total: 4min 20s
Wall time: 4min 56s


In [25]:
%%time

df = gpd.read_parquet('./trees.parquet', columns=['health', 'geometry'])

CPU times: user 365 ms, sys: 230 ms, total: 595 ms
Wall time: 570 ms


In [9]:
gdf.describe()

,health,geometry
count,652172,683788
unique,3,683352
top,Good,POINT (-73.76103529161729 40.660359381539436)
freq,528850,35


# GeoArrow & lonboard

In [10]:
import geoarrow.pyarrow as ga
import geoarrow.pandas as _

In [12]:
%%time
array = ga.as_geoarrow(gdf.geometry)

CPU times: user 153 ms, sys: 25.5 ms, total: 179 ms
Wall time: 199 ms


In [13]:
array

PointArray:PointType(geoarrow.point <PROJJSON:{"$schema":"https://proj....>)[683788]
<POINT (-73.84421521958048 40.723091773924274)>
<POINT (-73.81867945834878 40.79411066708779)>
<POINT (-73.93660770459083 40.717580740099116)>
<POINT (-73.93445615919741 40.713537494833226)>
<POINT (-73.97597938483258 40.66677775537875)>
...683778 values...
<POINT (-73.95494401022562 40.7132107823145)>
<POINT (-73.85665019989099 40.71519444267162)>
<POINT (-74.13651724205825 40.62076152739799)>
<POINT (-73.90311472453581 40.850828186655754)>
<POINT (-73.78752645502483 40.73216525220126)>

In [31]:
%pip install lonboard -q


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from IPython.core.display import display, HTML

/var/folders/wj/5qrl32td65j7gd1_kt8yrzd00000gn/T/ipykernel_97195/2058709175.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


⚠️ You may need to refresh your notebook after this to make sure the map loads

In [16]:
%%time

from lonboard import viz

viz(gdf)

CPU times: user 243 ms, sys: 39.1 ms, total: 282 ms
Wall time: 316 ms


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

# Spatial Index & Join Predicate

In [27]:
bgs = gpd.read_file('2010 Census Blocks.geojson')

In [38]:
bgs.head()

,shape_area,cb2010,bctcb2010,shape_leng,boro_name,ct2010,boro_code,geometry
0,244589.571777,1000,50009001000,2508.94793591,Staten Island,000900,5,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ..."
1,111006.257729,1000,50020011000,1345.88642116,Staten Island,002001,5,"MULTIPOLYGON (((-74.07062 40.61086, -74.07164 ..."
2,150406.766022,1000,50027001000,1703.38117241,Staten Island,002700,5,"MULTIPOLYGON (((-74.07524 40.62601, -74.07750 ..."
3,141296.561149,1000,50040001000,1511.17374266,Staten Island,004000,5,"MULTIPOLYGON (((-74.08709 40.61117, -74.08795 ..."
4,200784.982138,1000,50064001000,1978.24389924,Staten Island,006400,5,"MULTIPOLYGON (((-74.07644 40.60100, -74.07670 ..."


In [29]:
%%time
join = gdf.sjoin(bgs)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/geodataframe.py:2187: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: OGC:CRS84
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


CPU times: user 1.46 s, sys: 187 ms, total: 1.65 s
Wall time: 1.74 s


In [28]:
%%time
join = gdf.sjoin(bgs, predicate='contains')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/geodataframe.py:2187: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: OGC:CRS84
Right CRS: EPSG:4326

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


CPU times: user 522 ms, sys: 55.2 ms, total: 578 ms
Wall time: 632 ms


In [30]:
gdf.has_sindex

False

In [32]:
gdf.sindex

In [33]:
gdf.has_sindex

True

In [35]:
%%time
join = gdf.geometry.sindex.query(bgs.geometry, predicate='contains')

CPU times: user 562 ms, sys: 19.8 ms, total: 582 ms
Wall time: 604 ms


In [36]:
bgs.sindex

In [37]:
bgs.has_sindex

True

In [38]:
%%time
join = bgs.sjoin(df)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/geodataframe.py:2187: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: OGC:CRS84

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


CPU times: user 1.01 s, sys: 216 ms, total: 1.22 s
Wall time: 1.35 s


In [41]:
join.head()

,shape_area,cb2010,bctcb2010,shape_leng,boro_name,ct2010,boro_code,geometry,index_right,health
0,244589.571777,1000,50009001000,2508.94793591,Staten Island,000900,5,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",604321,Good
0,244589.571777,1000,50009001000,2508.94793591,Staten Island,000900,5,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",599063,Good
0,244589.571777,1000,50009001000,2508.94793591,Staten Island,000900,5,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",233374,Good
0,244589.571777,1000,50009001000,2508.94793591,Staten Island,000900,5,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",331024,Good
0,244589.571777,1000,50009001000,2508.94793591,Staten Island,000900,5,"MULTIPOLYGON (((-74.07921 40.64343, -74.07914 ...",329004,Good


In [40]:
%%time
join = bgs.sjoin(gdf, predicate='contains')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/geopandas/geodataframe.py:2187: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: OGC:CRS84

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)  # noqa: B026


CPU times: user 1.01 s, sys: 204 ms, total: 1.22 s
Wall time: 1.4 s


In [43]:
%%time
join = bgs.geometry.sindex.query(gdf.geometry, predicate='contains')

CPU times: user 432 ms, sys: 64.6 ms, total: 497 ms
Wall time: 530 ms


In [59]:
join

array([[     0,      1,      2, ..., 683785, 683786, 683787],
       [ 20912,  20658,  10029, ...,   2141,  12078,  16808]])